In [1]:
!pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list
! kaggle competitions download -c html2023-spring-final-project

Saving kaggle.json to kaggle.json
ref                                                       title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
arnabchaki/data-science-salaries-2023                     Data Science Salaries 2023 💸                       25KB  2023-04-13 09:55:16          14719        395  1.0              
chitrakumari25/corona-virus-latest-data-2023              Corona virus latest data 2023                      10KB  2023-04-29 16:00:51           1322         35  1.0              
utkarshx27/starbucks-nutrition                            Starbucks Nutrition Facts                           2KB  2023-05-10 05:42:59            804         32  1.0              
utkarshx27/monthly-transportation-statistics              Monthly 

In [2]:
! mkdir html_final
! unzip html2023-spring-final-project.zip -d html_final
! cd html_final

Archive:  html2023-spring-final-project.zip
  inflating: html_final/sample_submission.csv  
  inflating: html_final/test.csv     
  inflating: html_final/train.csv    


In [5]:
cd html_final/

/content/html_final


In [6]:
import csv
with open('train.csv',newline='') as csvfile:
  train = csv.reader(csvfile)
  train = list(train)

In [7]:
import numpy as np
lost = []
for i in range(len(train[0])):
  lost.append(0)
for i in range(1,len(train)):
  loss = 0
  for j in range(len(train[i])):
    if(train[i][j]==''):
      loss = loss + 1
  lost[loss] = lost[loss] + 1
lost = np.array(lost)
print(np.sum(lost))
print(lost)

17170
[ 205 1040 2370 3426 3522 2981 1901 1067  419  168   52   15    3    0
    1    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]


In [22]:
pick = [1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,23]
#可以選要用那些特徵，0是我們要predict的
#只選數值或true fale 但且沒有選id
feature = []
for i in range(1,len(train)):
  feature.append([])
  for j in pick:
    feature[i-1].append(train[i][j])

for i in range(len(feature)):
  for j in range(len(feature[i])):
    if(feature[i][j]!='' and feature[i][j]!='False' and feature[i][j]!= 'True'):
      feature[i][j] = float(feature[i][j])
    elif(feature[i][j] == 'False'):
      feature[i][j] = 0
    elif(feature[i][j] == 'True'):
      feature[i][j] = 1
    else:
      feature[i][j] = np.nan
print(len(feature))

17170


In [9]:
!pip install ycimpute

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:

feature = np.array(feature)
from ycimpute.imputer import knnimput
feature = knnimput.KNN(k=4).complete(feature) 
##可以修改這裡 使用不同的data填補方式


Imputing row 1/17170 with 1 missing, elapsed time: 71.295
Imputing row 101/17170 with 1 missing, elapsed time: 71.313
Imputing row 201/17170 with 0 missing, elapsed time: 71.333
Imputing row 301/17170 with 2 missing, elapsed time: 71.351
Imputing row 401/17170 with 1 missing, elapsed time: 71.367
Imputing row 501/17170 with 1 missing, elapsed time: 71.385
Imputing row 601/17170 with 4 missing, elapsed time: 71.403
Imputing row 701/17170 with 2 missing, elapsed time: 71.422
Imputing row 801/17170 with 2 missing, elapsed time: 71.441
Imputing row 901/17170 with 3 missing, elapsed time: 71.459
Imputing row 1001/17170 with 0 missing, elapsed time: 71.478
Imputing row 1101/17170 with 0 missing, elapsed time: 71.494
Imputing row 1201/17170 with 2 missing, elapsed time: 71.515
Imputing row 1301/17170 with 3 missing, elapsed time: 71.534
Imputing row 1401/17170 with 3 missing, elapsed time: 71.552
Imputing row 1501/17170 with 3 missing, elapsed time: 71.570
Imputing row 1601/17170 with 3 missi

In [26]:
predict = []
for i in range(1,len(train)):
  predict.append(train[i][0])
predict = np.array(predict)

import lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd



X_train, X_val, y_train, y_val = train_test_split(feature, predict, test_size=0.2 ,random_state=100 )
train_data = lgb.Dataset(X_train, label=y_train)

val_data = lgb.Dataset(X_val, label=y_val)

params = {
    'task':'train',
    'boosting_type': 'gbdt',# 設置提升類型
    'metric_freq': 1,
    'num_trees': 1000,
    'num_leaves': 64,
    'learning_rate': 0.01,
    'lambda_l1': 0.1,
    'lambda_l2': 0.2,
    'objective': 'multiclass',  # 目標函式
    'num_class': 10,
    'feature_fraction': 0.5,# 建樹的特徵選擇比例
    'feature_fraction_seed': 6,
    'bagging_fraction': 0.5,# 建樹的樣本採樣比例
    'bagging_freq': 5,  # k 意味着每 k 次迭代執行bagging
    'is_unbalance': 'true', #數據集如果樣本不均衡，可以明顯提高準確率
    'verbose': 0
}
gbm = lgb.train(params, train_data  ,valid_sets=[val_data])



/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:177: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000449 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[1]	valid_0's multi_logloss: 2.29639
[2]	valid_0's multi_logloss: 2.2927
[3]	valid_0's multi_logloss: 2.28773
[4]	valid_0's multi_logloss: 2.28394
[5]	valid_0's multi_logloss: 2.28023
[6]	valid_0's multi_logloss: 2.27602
[7]	valid_0's multi_logloss: 2.2717
[8]	valid_0's multi_logloss: 2.268
[9]	valid_0's multi_logloss: 2.26489
[10]	valid_0's multi_logloss: 2.26149
[11]	valid_0's multi_logloss: 2.25803
[12]	valid_0's multi_logloss: 2.25509
[13]	valid_0's multi_logloss: 2.2519
[14]	valid_0's multi_logloss: 2.2484
[15]	valid_0's multi_logloss: 2.24429
[16]	valid_0's multi_logloss: 2.24052
[17]	valid_0's multi_logloss: 2.23715
[18]	valid_0's multi_logloss: 2.23452
[19]	valid_0's multi_logloss: 2.23074
[20]	valid_0's multi_logloss: 2.22732
[21]	valid_0's multi_loglo

In [29]:
with open('test.csv',newline='') as csvfile:
  test = csv.reader(csvfile)
  test = list(test)
pick = [0,1,2,3,4,5,6,7,8,9,10,11,12,14,15,22]
#可以選要用那些特徵，0是我們要predict的
#只選數值或true fale 但且沒有選id
feature = []
for i in range(1,len(test)):
  feature.append([])
  for j in pick:
    feature[i-1].append(test[i][j])

for i in range(len(feature)):
  for j in range(len(feature[i])):
    if(feature[i][j]!='' and feature[i][j]!='False' and feature[i][j]!= 'True'):
      feature[i][j] = float(feature[i][j])
    elif(feature[i][j] == 'False'):
      feature[i][j] = 0
    elif(feature[i][j] == 'True'):
      feature[i][j] = 1
    else:
      feature[i][j] = np.nan
print(len(feature))

feature = np.array(feature)
from ycimpute.imputer import knnimput
feature = knnimput.KNN(k=4).complete(feature)


y_pred = gbm.predict(feature)
y_pred = [list(x).index(max(x)) for x in y_pred]
print(y_pred)

6315
Imputing row 1/6315 with 1 missing, elapsed time: 8.873
Imputing row 101/6315 with 2 missing, elapsed time: 8.887
Imputing row 201/6315 with 3 missing, elapsed time: 8.905
Imputing row 301/6315 with 2 missing, elapsed time: 8.919
Imputing row 401/6315 with 4 missing, elapsed time: 8.931
Imputing row 501/6315 with 1 missing, elapsed time: 8.943
Imputing row 601/6315 with 1 missing, elapsed time: 8.955
Imputing row 701/6315 with 2 missing, elapsed time: 8.968
Imputing row 801/6315 with 3 missing, elapsed time: 8.981
Imputing row 901/6315 with 0 missing, elapsed time: 8.994
Imputing row 1001/6315 with 3 missing, elapsed time: 9.007
Imputing row 1101/6315 with 1 missing, elapsed time: 9.020
Imputing row 1201/6315 with 5 missing, elapsed time: 9.032
Imputing row 1301/6315 with 3 missing, elapsed time: 9.045
Imputing row 1401/6315 with 2 missing, elapsed time: 9.057
Imputing row 1501/6315 with 1 missing, elapsed time: 9.069
Imputing row 1601/6315 with 3 missing, elapsed time: 9.082
Impu

In [32]:
import csv
with open("predict.csv","w",newline="") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(["id","Danceability"])
  for i in range(len(y_pred)):
    writer.writerow([17170+i,y_pred[i]])

In [33]:
ls

predict  predict.csv  sample_submission.csv  test.csv  train.csv
